# Projeto 1 - Ciência dos Dados

Nome: Lorena Barbosa Antunes Da Silva

Nome: José Edson Mendonça Ribeiro Lima Araújo

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [24]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
!pip install emoji
import emoji

In [25]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\loren\Repositorios\CDados_Projeto1


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [26]:
filename = 'Peaky Blinders.xlsx'

In [27]:
train = pd.read_excel(filename)
linha=np.arange(0,300,1)
train=train.iloc[linha,[0,1]]
train.head(5)
#train.tail()

,Treinamento,Classificação
0,peaky fucking blinders foi a melhor coisa que ...,1.0
1,@limawt1 peaky blinders está vindo em breve!! ...,0.0
2,gente cancela ele me seguiu de outro instagram...,0.0
3,até eu que não sou fumante fico com vontade de...,0.0
4,tou a curtir bue de peaky blinders,1.0


In [28]:
test = pd.read_excel(filename, sheet_name = 'Teste')
linha=np.arange(0,200,1)
test=test.iloc[linha,[0,1]]
test.head(5)
test.tail(5)

,Teste,Classificação
195,viciada em peaky blinders 🖤,1.0
196,@pedropossobom peaky blinders mastermind \nmei...,1.0
197,@italoverdose peaky blinders está vindo em bre...,0.0
198,"a gente fuma tbem, somos os peaky blinders htt...",0.0
199,o lado bom de tá com covid e que eu tô maraton...,1.0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

ESCREVA AQUI...

Produto: Série Peaky Blinders, consideramos relevante qualquer comentário positivo ou negativo, relacionados a série mostrando que estão assistindo ou indicando. Exemplo: "Não curti assistir Peaky Blinders", "@user1 vc precisa assistir peaky blinders", "acabei de maratonar essa serie peaky blinders".

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [29]:
from IPython.display import display
pd.options.display.max_rows = 13


import re 



def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    limpa_emoji=' '.join(emoji.get_emoji_regexp().split(text))
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', limpa_emoji)
    
    return text_subbed

In [30]:
comentarios_relevantes=[]
comentarios_irrelevantes=[]
for i in range(0,300,1):
    comentario=cleanup(train.Treinamento[i]).lower().split()
    if train.Classificação[i]==1:
        comentarios_relevantes.append(comentario)
    elif train.Classificação[i]==0:
        comentarios_irrelevantes.append(comentario)


In [31]:
l_irrelevantes=[]
l_relevantes=[]
for comentario in comentarios_irrelevantes:
    lista_remocao=[]
    for palavra in comentario:
        if  palavra[0]=='@' or 'htt' == palavra[0:3]:
            lista_remocao.append(palavra)
    for palavra in lista_remocao:
        comentario.remove(palavra)
    for palavra in comentario:
        l_irrelevantes.append(palavra)
        
for comentario in comentarios_relevantes:
    lista_remocao=[]
    for palavra in comentario:
        if  palavra[0]=='@' or 'htt' == palavra[0:3]:
            lista_remocao.append(palavra)
    for palavra in lista_remocao:
        comentario.remove(palavra)

for comentario in comentarios_irrelevantes:
    for palavra in comentario:
        l_irrelevantes.append(palavra)
for comentario in comentarios_relevantes:
    for palavra in comentario:
        l_relevantes.append(palavra)

In [32]:
s_l_relevantes=  pd.Series(l_relevantes)

In [33]:
s_l_irrelevantes=  pd.Series(l_irrelevantes)

In [34]:
tab_relevantes = s_l_relevantes.value_counts()

In [35]:
tab_irrelevantes = s_l_irrelevantes.value_counts()

In [36]:
total = comentarios_relevantes + comentarios_irrelevantes
lista_total = l_irrelevantes + l_relevantes

In [37]:
s_l_total = pd.Series(lista_total)
tab_l_total= s_l_total.value_counts()

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [38]:
test_relevantes=[]
test_irrelevantes=[]
test_total = []

for i in range(0,200,1):
    comentarios_test=cleanup(test.Teste[i]).lower().split()
    
    if test.Classificação[i]==1:
        test_relevantes.append(comentarios_test)
        
    elif test.Classificação[i]==0:
        test_irrelevantes.append(comentarios_test)
        
    test_total.append(comentarios_test)


In [39]:
P_relevante = len(l_relevantes)/len(lista_total)
P_irrelevante = len(l_irrelevantes)/len(lista_total)
P_relevante + P_irrelevante

1.0

In [40]:
P_relevante_dado_comentario = []
P_irrelevante_dado_comentario = []
relevancia_classificador = []
i=0

while i < len(test_total): 
    P_comentario_dado_relevante = 1
    P_comentario_dado_irrelevante = 1
   
    for palavra in test_total[i]:
        if palavra not in l_relevantes:
            tab_relevantes[palavra]=0
            
        if palavra not in l_irrelevantes:
            tab_irrelevantes[palavra]=0
            
        P_comentario_dado_relevante *= (tab_relevantes[palavra]+1)/(len(l_relevantes)+tab_l_total.size)
        P_comentario_dado_irrelevante *= (tab_irrelevantes[palavra]+1)/(len(l_irrelevantes)+tab_l_total.size)
    
    P_relevante_dado_comentario.append(P_comentario_dado_relevante*P_relevante)
    P_irrelevante_dado_comentario.append(P_comentario_dado_irrelevante*P_irrelevante)
    
    if P_relevante_dado_comentario[i] > P_irrelevante_dado_comentario[i]:
        relevancia_classificador.append(1)
        
    else:
        relevancia_classificador.append(0)
        
    i += 1

In [41]:
test["Relevancia_Classificador"]=relevancia_classificador

In [42]:
Matriz_de_Comparação = pd.crosstab(test["Classificação"] , test["Relevancia_Classificador"], normalize=True)*100
Matriz_de_Comparação

Relevancia_Classificador,0,1
Classificação,,
0.0,20.5,34.5
1.0,3.0,42.0


In [43]:
print("Acurácia:{0:.2f}%".format(Matriz_de_Comparação[0][0] + Matriz_de_Comparação[1][1]))

Acurácia:62.50%


___
### Concluindo

Nosso classificador teve uma **acurácia de 62,50%**, o que significa que ele classifica de forma adequada **62,50%** dos tweets analisados.
* Observou-se um percentual de **42,00% verdadeiros positivos**, ou seja, aproximadamente **42,00%** dos comentários relevantes foram corretamente classificados como relevantes. 
* Observou-se um percentual de **34,50% falsos positivos**, ou seja, aproximadamente **34,50%** dos comentários irrelevantes foram incorretamente classificados como relevantes. 
* Observou-se um percentual de **20,50% verdadeiros negativos**, ou seja, aproximadamente **20,50%** dos comentários irrelevantes foram corretamente classificados como irrelevantes. 
* Observou-se um percentual de **3,00% falsos negativos**, ou seja, aproximadamente **3,00%** dos comentários relevantes foram incorretamente classificados como irrelevantes. 

##### Sarcasmo e dupla negação

Neste projeto, com o classificador Naïve-Bayes, assumiu-se a independência entre as palavras, considerando-se apenas a probabilidade destas de aparecerem em determinada categoria. Ou seja, o classificador não leva em consideração as relações entre palavras e a sintaxe da língua portuguesa. O que pode levar à uma classificação errônea de frases sarcásticas, ou com dupla negação.

In [70]:
frase_sarcastica = "Peaky Binders é tão bom quanto tomar chuva"

In [71]:
frase_duplanegacao = "Eu nunca não vou assistir Peaky Blinders"

##### Plano de Expansão

Tendo em considereção que o projeto já apresentou 62,5% de acurácia. Para que o classificador fosse mais preciso poder-se-iam fazer futuras iterações que justificariam um maior investimento no projeto, como uma limpeza adicional dos tweets obtidos; levar em conta a sintaxe e organização entre palavras da língua portuguesa, ou seja, não admitir que as palavras têm probabilidade de aparecer em uma frase independentemente.

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

In [121]:
def item6(repeticoes):
    train_dados6 = pd.read_excel(filename)
    linha=np.arange(0,300,1)
    train_dados6=train_dados6.iloc[linha,[0,1]]
    test_dados6 = pd.read_excel(filename, sheet_name = 'Teste')
    linha=np.arange(0,200,1)
    test_dados6=test_dados6.iloc[linha,[0,1]]
    for contador_item6 in range(0, repeticoes):
        test_dados6 = test_dados6.rename(columns={"Teste":"Treinamento"})
        total=pd.concat([train_dados6,test_dados6], ignore_index = True)
        treino_dados = total.sample(300)
        teste_dados = total.drop(treino_dados.index)
        teste_dados = teste_dados.rename(columns = {"Treinamento":"Teste"})

        
        relevantes_treino6=[]
        irrelevantes_treino6=[]
        for i in range(0,300,1):
            comentario6_treino=cleanup(treino_dados.Treinamento[i]).lower().split()
            if treino_dados.Classificação[i]==1:
                relevantes_treino6.append(comentario6_treino)
            elif treino_dados.Classificação[i]==0:
                irrelevantes_treino6.append(comentario6_treino)
        
        relevantes_teste6=[]
        irrelevantes_teste6 = []
        total_teste_6 = []
        for j in range(0,200,1):
            comentario6_teste=cleanup(teste_dados.Teste[j]).lower().split()
            if teste_dados.Classificação[j]==1:
                relevantes_teste6.append(comentario6_teste)
            elif teste_dados.Classificação[j]==0:
                irrelevantes_teste6.append(comentario6_teste)
                
            total_teste6.append(comentarios6_teste)
                
        l_irrelevantes_treino6=[]
        l_relevantes_treino6=[]
        for comentario in irrelevantes_treino6:
            lista_remocao=[]
        for palavra in comentario:
            if  palavra[0]=='@' or 'htt' == palavra[0:3]:
                lista_remocao.append(palavra)
        for palavra in lista_remocao:
            comentario.remove(palavra)
        for palavra in comentario:
            l_irrelevantes_treino6.append(palavra)
        
        for comentario in relevantes_treino6:
            lista_remocao=[]
        for palavra in comentario:
            if  palavra[0]=='@' or 'htt' == palavra[0:3]:
                lista_remocao.append(palavra)
        for palavra in lista_remocao:
            comentario.remove(palavra)

        for comentario in irrelevantes_treino6:
            for palavra in comentario:
                l_irrelevantes_treino6.append(palavra)
        for comentario in relevantes_treino6:
            for palavra in comentario:
                l_relevantes_treino6.append(palavra)
                
        s_l_relevantes_treino6=  pd.Series(l_relevantes_treino6)
        s_l_irrelevantes_treino6=  pd.Series(l_irrelevantes_treino6)
        
        tab_relevantes_treino6 = s_l_relevantes_treino6.value_counts()
        tab_irrelevantes_treino6 = s_l_irrelevantes_treino.value_counts()
        
        lista_total = l_irrelevantes_treino6 + l_relevantes_treino6
        
        s_l_total = pd.Series(lista_total)
        tab_l_total= s_l_total.value_counts()
        
        
        P_relevante6 = len(l_relevantes_treino6)/len(lista_total)
        P_irrelevante6 = len(l_irrelevantes_treino6)/len(lista_total)
        
        P_relevante_dado_comentario6 = []
        P_irrelevante_dado_comentario6 = []
        relevancia_classificador6 = []
        ki=0

        while k < len(total_teste6): 
            P_comentario_dado_relevante6 = 1
            P_comentario_dado_irrelevante6 = 1
   
            for palavra in total_teste6[k]:
                if palavra not in l_relevantes_treino6:
                    tab_relevantes_treino6[palavra]=0
            
                if palavra not in l_irrelevantes_treino6:
                    tab_irrelevantes_treino6[palavra]=0
            
                P_comentario_dado_relevante6 *= (tab_relevantes_treino6[palavra]+1)/(len(l_relevantes_treino6)+tab_l_total.size)
                P_comentario_dado_irrelevante6 *= (tab_irrelevantes_treino6[palavra]+1)/(len(l_irrelevantes_treino6)+tab_l_total.size)
    
            P_relevante_dado_comentario6.append(P_comentario_dado_relevante6*P_relevante6)
            P_irrelevante_dado_comentario6.append(P_comentario_dado_irrelevante6*P_irrelevante6)
    
            if P_relevante_dado_comentario6[k] > P_irrelevante_dado_comentario6[k]:
                relevancia_classificador.append(1)
        
            else:
                relevancia_classificador.append(0)
        
            k += 1
            
            teste_dados["Relevancia_Classificador"]=relevancia_classificador
            Matriz_de_Comparação6 = pd.crosstab(teste_dados["Classificação"] , teste_dados["Relevancia_Classificador"], normalize=True)*100
            
            acertos[contador_item6] = Matriz_de_Comparação6[0][0]+Matriz_de_Comparação6[1][1]
            

In [122]:
item6(2)

KeyError: 2

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**